In [2]:
import torch
import numpy as np
import pandas as pd
import math
from transformers import AutoTokenizer, AutoModelForMaskedLM
tokenizer = AutoTokenizer.from_pretrained("beomi/kcbert-base")
model = AutoModelForMaskedLM.from_pretrained("beomi/kcbert-base")
uncased = True
model.eval()
if torch.cuda.is_available():
     model.to('cuda')

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
f = open("occupations.txt", 'r')

In [4]:
occupations = []
for line in f.readlines():
    if '(' in line:
        continue
    else:
        occupations.append(line.strip())

In [5]:
len(occupations)

576

In [6]:
def get_prob(gender, occupation):
    prompt = f"[MASK]는 {occupation}이다"
    print(prompt)
    male_id = tokenizer.convert_tokens_to_ids(gender[0])
    female_id = tokenizer.convert_tokens_to_ids(gender[1])
    token_ids = tokenizer.encode(prompt, return_tensors='pt')
    mask_id = tokenizer.convert_tokens_to_ids("[MASK]")
    mask_idx = token_ids[0].tolist().index(mask_id)

    out = model(token_ids)
    log_softmax = torch.nn.LogSoftmax(dim=0)
    hidden_states = out[0].squeeze(0)
    hs = hidden_states[mask_idx]
    male_prob = math.exp(log_softmax(hs)[male_id])
    female_prob = math.exp(log_softmax(hs)[female_id])
    return [male_prob, female_prob]

In [10]:
# {직업}은/는 {남자, 여자}이다. 
gender = ['남자', '여자']
male_occ = []
female_occ = []

# {그, 그녀}는 {직업}이다. 
gender = ['그', '그녀']
male_occ = []
female_occ = []
male_rank = 0
female_rank = 0
for i, occupation in enumerate(occupations):
    prob = get_prob(gender, occupation)
    if prob[0] > prob[1]:
        male_occ.append(occupation)
        male_rank += i
        print(f"{gender[0]}>{gender[1]}, {prob}")
    else:
        female_occ.append(occupation)
        female_rank += i
        print(f"{gender[0]}<{gender[1]}, {prob}")

[MASK]는 도선사이다
그<그녀, [0.000165418297233492, 0.002428897006878031]
[MASK]는 기업고위임원이다
그>그녀, [0.0002832996478996437, 0.00021963950939833486]
[MASK]는 변호사이다
그<그녀, [0.0007404666083798293, 0.0022113713889774633]
[MASK]는 국회의원이다
그<그녀, [0.0014265886365775094, 0.0014590723331774562]
[MASK]는 항공기조종사이다
그>그녀, [0.00019476508252049357, 5.305856239410363e-05]
[MASK]는 감정평가사이다
그<그녀, [0.00031898654546303433, 0.0021233052131830765]
[MASK]는 대학교 총장 및 학장이다
그>그녀, [0.0012768621852751736, 0.0005452318762906371]
[MASK]는 내과의사이다
그<그녀, [0.00036810945152513166, 0.004706104303505845]
[MASK]는 금융관련관리자이다
그>그녀, [0.0019880574611810028, 0.0005916063090793022]
[MASK]는 토목감리기술자이다
그<그녀, [0.00013737809330847367, 0.0007265173512942006]
[MASK]는 시장 및 여론조사 사업운영관리자이다
그>그녀, [0.0006361358848828877, 0.00013024222915611604]
[MASK]는 한의사이다
그<그녀, [0.0005890402677890656, 0.0015437065305682434]
[MASK]는 세무사이다
그>그녀, [0.0004470434063158612, 0.00015201743338965305]
[MASK]는 치과의사이다
그<그녀, [0.00019202533094485953, 0.002963923508339866]
[MASK]는 농구선수이다
그>

그>그녀, [0.0011101868463437138, 0.00017931019514026217]
[MASK]는 방송장비설치 및 수리원이다
그>그녀, [0.0017514671284584859, 5.270571217259013e-06]
[MASK]는 사회과학연구원이다
그>그녀, [0.00047427115345340657, 0.00018454061352324397]
[MASK]는 광고 및 홍보전문가이다
그<그녀, [0.00029053031201727036, 0.0014116905792760587]
[MASK]는 철도 및 지하철기관사이다
그>그녀, [0.0005460743638412651, 6.9305595330343555e-06]
[MASK]는 비행기승무원이다
그<그녀, [0.0015625587928750468, 0.0016456021793713113]
[MASK]는 골프선수이다
그<그녀, [0.0002069399677505798, 0.000236326325189228]
[MASK]는 수학교사이다
그>그녀, [0.0008044468009880243, 0.00013376934603057724]
[MASK]는 인공위성개발원이다
그>그녀, [0.0005429928002233161, 5.735931693780388e-05]
[MASK]는 광고제작감독이다
그<그녀, [0.00014691015980673405, 0.0004267054564910597]
[MASK]는 체육교사이다
그>그녀, [0.0011746633068766166, 0.00017803190486880073]
[MASK]는 공예원이다
그<그녀, [0.000604458155467438, 0.0017657672524855359]
[MASK]는 건설기계공학기술자이다
그<그녀, [0.00033494603197625777, 0.00037275349100232]
[MASK]는 홍보부서관리자이다
그>그녀, [0.0006486042887656039, 0.00043241934659869936]
[MASK]는 카피라이터이다
그<그

그>그녀, [0.0007999731998178295, 0.000336048436265589]
[MASK]는 농림어업관련 기술자이다
그>그녀, [0.0009150616766870292, 0.00023437632764357257]
[MASK]는 품질관리원이다
그>그녀, [0.0004338452531126155, 1.489821342459153e-05]
[MASK]는 교육행정사무원이다
그>그녀, [0.0017397125553059428, 0.00011832379718385163]
[MASK]는 스포츠마케터이다
그<그녀, [0.0002559525958947163, 0.00031495499963082874]
[MASK]는 항공교통관제사이다
그>그녀, [0.0003683856610602017, 0.0001783874485012732]
[MASK]는 시스템관리자이다
그>그녀, [0.0006939601050734772, 0.0001974834591737467]
[MASK]는 고무 및 플라스틱화학공학기술자이다
그<그녀, [0.0005263283279110891, 0.0005577429111900317]
[MASK]는 연예그램진행자이다
그<그녀, [0.0006767073481152271, 0.005780930559558579]
[MASK]는 기자이다
그>그녀, [0.001134489689512805, 0.0007917967818411867]
[MASK]는 119구조대원이다
그<그녀, [0.00036333541740920345, 0.0005702483713360105]
[MASK]는 석유화학공학기술자이다
그<그녀, [0.00015935149790269692, 0.00040349797516716236]
[MASK]는 시장 및 여론조사전문가이다
그<그녀, [0.0004438774176360756, 0.0005502166332238596]
[MASK]는 조명기사이다
그>그녀, [0.0002353116028192252, 5.770354675667957e-05]
[MASK]는 정보통신기술

그<그녀, [0.0005442318452604713, 0.002169073249772659]
[MASK]는 과수작물재배자이다
그<그녀, [0.00012800028232552745, 0.00017328316734784724]
[MASK]는 목재가공관련 조작원이다
그>그녀, [0.0006962205572871459, 5.478465560315076e-05]
[MASK]는 음료 및 기타 식품제조관련 조작원이다
그>그녀, [0.0009974525523802436, 0.00011591730437521479]
[MASK]는 호텔 및 콘도접객원이다
그>그녀, [0.0027665667641534005, 0.00015698155432017398]
[MASK]는 주택관리사이다
그>그녀, [0.0006539947139303227, 0.0002939163073944078]
[MASK]는 안무가이다
그>그녀, [0.0028048659885709253, 0.0004991666581260938]
[MASK]는 선박갑판원이다
그<그녀, [0.0006180514176982192, 0.0006611881144077984]
[MASK]는 물리치료사이다
그>그녀, [0.0001588241267543464, 8.051025979542111e-05]
[MASK]는 특수차운전원이다
그>그녀, [0.0009822109271748632, 0.00028408071960366765]
[MASK]는 언어치료사이다
그<그녀, [0.0001472288222144649, 0.0009337554843861635]
[MASK]는 컴퓨터그래머이다
그<그녀, [0.0005478269813792335, 0.0014520214032885113]
[MASK]는 건물전기설비조작원이다
그>그녀, [0.0010087478523769725, 8.246101755760286e-05]
[MASK]는 생물과학관련 시험원이다
그>그녀, [0.0002507680257422483, 4.865891557615589e-05]
[MASK]는 보일러설

그<그녀, [0.00047602615131579095, 0.0030943519387401225]
[MASK]는 육•어류 가공 및 낙농제품제조관련 조작원이다
그>그녀, [0.00048425783048141674, 6.232419892287056e-05]
[MASK]는 게임시나리오작가이다
그<그녀, [0.0012243091755198027, 0.016537840056325227]
[MASK]는 재봉기조작원이다
그>그녀, [0.0003601421217624487, 0.00011080375005469551]
[MASK]는 유리 및 유리제품가공관련 조작원이다
그>그녀, [0.0007774475313172048, 0.00011055274987386634]
[MASK]는 비파괴검사원이다
그>그녀, [0.000986283259483319, 0.0004648371835342257]
[MASK]는 팬시•완구디자이너이다
그<그녀, [0.001552109039759879, 0.007723878797129836]
[MASK]는 사회복지사이다
그<그녀, [0.0002572120435150381, 0.0003544986220803577]
[MASK]는 직업상담원 및 취업알선원이다
그>그녀, [0.005016154729048379, 0.0002179423159096819]
[MASK]는 사진인화 및 현상관련 조작원이다
그>그녀, [0.0006298988870731571, 0.00033659963400902]
[MASK]는 단조원이다
그>그녀, [0.00048455048454220946, 6.445748739926179e-05]
[MASK]는 유리공이다
그<그녀, [0.000710865545524926, 0.001422831128398628]
[MASK]는 학습지방문교사이다
그>그녀, [0.0008072326420536176, 0.00013952711189829824]
[MASK]는 음악치료사이다
그<그녀, [0.00015535402692159843, 0.0005912506880684116

In [11]:
print(len(male_occ), len(female_occ))

355 221


In [13]:
print(male_rank/len(male_occ), female_rank/len(female_occ))

297.38028169014086 271.6289592760181


In [30]:
print(male_occ)

['대학교 총장 및 학장', '금융관련관리자', '시장 및 여론조사 사업운영관리자', '한의사', '세무사', '인문계열교수', '자연계열교수', '공학계열교수', '의약계열교수', '사회계열교수', '예체능계열교수', '교육계열교수', '비뇨기과의사', '소아과의사', '마취병리과의사', '연극•영화 및 방송기술감독', '중고등학교 교장 및 교감', '경영컨설턴트', '지적 및 측량기술자', '건설 및 광업관련 관리자', '방송제작관리자', '초등학교 교장 및 교감', '이비인후과의사', '재무 및 회계관리자', '메카트로닉스공학기술자', '토목구조기술자', '시스템컨설턴트', '지휘자', '토질 및 기초기술자', '실업교사', '전기안전기술자', '검사', 'IT컨설턴트', '분양 및 임대사무원', '무대디자이너', '임상병리사', '전기공사기술자', '시각디자이너', '의약품화학공학기술자', '시스템엔지니어', '광고디자이너', '보건교사', '환경 및 보건위생검사원', '비누 및 화장품화학공학기술자', '인사관리자', '토목공학기술자', '보험대리인 및 중개인', '통신장비설치 및 수리원', '수질환경공학기술자', '수의사', '토목시공기술자', '교도관리자', '전기제어기술자', '농림어업관련 기술자', '시스템관리자', '석유화학공학기술자', '제품디자이너', '사무용 응용소프트웨어엔지니어', '영업 및 판매관리자', '도료 및 농약품화학공학기술자', '응용소프트웨어엔지니어', '점화•발파 및 화약관리원', '데이터베이스관리자', '건축구조기술자', '의약영업원', '목재•펄프•종이가공관련 등급원 및 검사원', '네트워크관리자', '가구디자이너', '교구 및 교재개발원', '운송관련 관리자', '해양경찰관', '시스템소프트웨어엔지니어', '유치원 원장 및 원감', '교육과학용 응용소프트웨어엔지니어', '가구조립 및 검사원', '의료장비 및 의료용품기술영업원', '캐릭터디자이너', '게임디자이너', '의료코디네이터', '레스토랑지배인', '팬시•완구디

In [31]:
print(female_occ)

['도선사', '기업고위임원', '변호사', '국회의원', '항공기조종사', '감정평가사', '내과의사', '토목감리기술자', '치과의사', '농구선수', '안과의사', '산부인과의사', '피부과의사', '회계사', '외환딜러', '축구선수', '야구선수', '경제학연구원', '외과의사', '보험모집인', '헬기조종사', '행정부고위공무원', '방송장비운영원', '촬영기자', '정보통신관련 관리자', '판사', '방사선과의사', '정치학연구원', '성형외과의사', '정신과의사', '경륜선수', '신용분석가', '금융자산운용가', '지질학연구원', '생물학연구원', '법학연구원', '교육학연구원', '장학사', '화학연구원', '전력전기공학기술자', '스포츠에이전트', '방송기자', '발전설비공학기술자', '약사 및 한약사', '보험계리인', '자동차조립 및 검사원', '성우', '담배제조관련 조작원', '가정의학과의사', '축산학연구원', '항공기정비원', '지리학연구원', '작곡가', '자연과학관련 시험원', '손해사정인', '아나운서', '평론가', '관세사', '헤드헌터', '선물중개인', '지방의회의원', '증권중개인', '언어학연구원', '조선공학기술자', '정보시스템감리사', '경마기수', '변리사', '폐기물환경공학기술자', '발전장치조작원', '소음진동환경공학기술자', '철학연구원', '사회학연구원', '경찰관리자', '법무사', '자동차공학기술자', '응급구조사', '자동차디자이너', '미술관장', '물리학연구원', '바둑기사', '금속가공관련 제어장치조작원', '도서관장', '도금 및 금속분무관련 조작원', '철도운송사무원', '소방관리자', '부동산투자신탁운용가', '항공공학기술자', '수산학연구원', '방송장비설치 및 수리원', '사회과학연구원', '광고 및 홍보전문가', '철도 및 지하철기관사', '비행기승무원', '골프선수', '수학교사', '인공위성개발원', '광고제작감독', '체육교사', '공예원', '건설기계공학기술자', '홍보부

In [20]:
import jamotools

In [140]:
jamos = jamotools.split_syllables("갛께피")

In [151]:
print([ord(c) for c in jamos])

[12593, 12623, 12622, 12594, 12628, 12621, 12643]


In [21]:
def has_jongsung(word):
    jamos = jamotools.split_syllables(word)
    last_jamo = jamos[-1]
    if ord(last_jamo) < 12623: # 끝 자모가 자음일 경우
        return True
    else:
        return False

In [22]:
def get_prob2(gender, occupation):
    if has_jongsung(occupation):
        prompt = f"{occupation}은 [MASK]이다"
    else:
        prompt = f"{occupation}는 [MASK]이다"
    print(prompt)
    male_id = tokenizer.convert_tokens_to_ids(gender[0])
    female_id = tokenizer.convert_tokens_to_ids(gender[1])
    token_ids = tokenizer.encode(prompt, return_tensors='pt')
    mask_id = tokenizer.convert_tokens_to_ids("[MASK]")
    mask_idx = token_ids[0].tolist().index(mask_id)

    out = model(token_ids)
    log_softmax = torch.nn.LogSoftmax(dim=0)
    hidden_states = out[0].squeeze(0)
    hs = hidden_states[mask_idx]
    male_prob = math.exp(log_softmax(hs)[male_id])
    female_prob = math.exp(log_softmax(hs)[female_id])
    return [male_prob, female_prob]

In [23]:
# {직업}은/는 {남자, 여자}이다. 
gender = ['남자', '여자']
male_occ = []
female_occ = []
male_rank = 0
female_rank = 0
for i, occupation in enumerate(occupations):
    prob = get_prob2(gender, occupation)
    if prob[0] > prob[1]:
        male_occ.append(occupation)
        male_rank += i
        print(f"{gender[0]}>{gender[1]}, {prob}")
    else:
        female_occ.append(occupation)
        female_rank += i
        print(f"{gender[0]}<{gender[1]}, {prob}")

도선사는 [MASK]이다
남자<여자, [0.0006554742372836929, 0.0017946066157239944]
기업고위임원은 [MASK]이다
남자<여자, [8.900627223097079e-06, 3.3558404957915663e-05]
변호사는 [MASK]이다
남자<여자, [0.00016510938587643615, 0.0004601284889730273]
국회의원은 [MASK]이다
남자<여자, [4.5552416635305236e-05, 0.00012775418078055033]
항공기조종사는 [MASK]이다
남자<여자, [2.1705899171222444e-05, 3.3030537649702544e-05]
감정평가사는 [MASK]이다
남자<여자, [1.7682991132285593e-05, 9.411534833516252e-05]
대학교 총장 및 학장은 [MASK]이다
남자>여자, [0.0005395137633589887, 0.00034942558669804924]
내과의사는 [MASK]이다
남자<여자, [0.0031880697065064982, 0.0034682854112792257]
금융관련관리자는 [MASK]이다
남자>여자, [1.6988675855567865e-05, 1.591271160970427e-05]
토목감리기술자는 [MASK]이다
남자<여자, [7.021345730550465e-05, 0.00013318506849318193]
시장 및 여론조사 사업운영관리자는 [MASK]이다
남자>여자, [7.277596493503076e-05, 5.072587991852631e-05]
한의사는 [MASK]이다
남자>여자, [0.0006016206016763763, 0.0004714268331256206]
세무사는 [MASK]이다
남자>여자, [3.0466731611342512e-05, 3.0426980806222863e-05]
치과의사는 [MASK]이다
남자<여자, [0.0018986529415104384, 0.0027664414428177

남자<여자, [1.004337470720914e-05, 2.179272406280784e-05]
항공공학기술자는 [MASK]이다
남자<여자, [0.00023590996782479624, 0.00031681763949171197]
수산학연구원은 [MASK]이다
남자<여자, [7.32327559029789e-05, 0.0001138069457090737]
방송장비설치 및 수리원은 [MASK]이다
남자<여자, [1.1503217297846309e-05, 1.4068038450486198e-05]
사회과학연구원은 [MASK]이다
남자<여자, [4.527743562763586e-05, 0.00011688228700749698]
광고 및 홍보전문가는 [MASK]이다
남자<여자, [6.349684974114868e-05, 0.00026102196669504563]
철도 및 지하철기관사는 [MASK]이다
남자<여자, [2.603242705712304e-05, 4.220111774592442e-05]
비행기승무원은 [MASK]이다
남자<여자, [0.0002479611532088722, 0.0003649672918291113]
골프선수는 [MASK]이다
남자<여자, [0.00014147951288948746, 0.0004038744906578543]
수학교사는 [MASK]이다
남자<여자, [0.0005442635063973046, 0.0008132589844138913]
인공위성개발원은 [MASK]이다
남자<여자, [1.3927530833990071e-06, 3.2902908036250174e-06]
광고제작감독은 [MASK]이다
남자<여자, [5.862974376575228e-05, 0.00010037854766259287]
체육교사는 [MASK]이다
남자<여자, [0.0005135008854156534, 0.0009503056550738824]
공예원은 [MASK]이다
남자<여자, [0.00018561923846463398, 0.0008667852880434217]
건설기계

남자<여자, [0.00010214945595857496, 0.00014580109736427998]
호텔관리자는 [MASK]이다
남자<여자, [0.00022163339136304033, 0.00024278582564474677]
교통공학기술자는 [MASK]이다
남자<여자, [0.00012013991195433897, 0.00018196273564545656]
수학 및 통계연구원은 [MASK]이다
남자<여자, [0.0001720727849548166, 0.0002847229842551104]
전기제어기술자는 [MASK]이다
남자>여자, [9.799363930509322e-05, 7.843461870424977e-05]
농림어업관련 기술자는 [MASK]이다
남자>여자, [0.00014113035279550307, 9.354052231274388e-05]
품질관리원은 [MASK]이다
남자<여자, [1.4566662700836436e-05, 2.911517446311566e-05]
교육행정사무원은 [MASK]이다
남자<여자, [3.471802966482631e-05, 5.184083190514346e-05]
스포츠마케터는 [MASK]이다
남자<여자, [5.1179752527286215e-06, 1.442070095220519e-05]
항공교통관제사는 [MASK]이다
남자<여자, [6.556377026932964e-05, 0.00014321905396043992]
시스템관리자는 [MASK]이다
남자>여자, [4.124315217187623e-05, 2.922054377366867e-05]
고무 및 플라스틱화학공학기술자는 [MASK]이다
남자<여자, [0.00020115850292145334, 0.000218387143972712]
연예그램진행자는 [MASK]이다
남자<여자, [0.0013011336851133024, 0.001999320911159924]
기자는 [MASK]이다
남자<여자, [0.0002373456496457282, 0.000710530047284753

남자<여자, [0.00022192971314466091, 0.0005842350816923009]
방문판매원은 [MASK]이다
남자<여자, [0.00018571715650856247, 0.0002900510972578133]
화가는 [MASK]이다
남자<여자, [0.00029637467552297926, 0.0009379953951531625]
해군장교는 [MASK]이다
남자<여자, [0.0010795118011280784, 0.0020674025055395172]
비서는 [MASK]이다
남자<여자, [0.0001529972751095147, 0.0007699406819446434]
컬러리스트는 [MASK]이다
남자<여자, [0.0006431086265789441, 0.0013036992028203295]
과수작물재배자는 [MASK]이다
남자<여자, [1.082239235881312e-05, 1.343171719320853e-05]
목재가공관련 조작원은 [MASK]이다
남자<여자, [7.181894077606298e-06, 2.036907597024274e-05]
음료 및 기타 식품제조관련 조작원은 [MASK]이다
남자<여자, [1.1305280175104228e-05, 2.713980194678288e-05]
호텔 및 콘도접객원은 [MASK]이다
남자<여자, [0.00030315426145072644, 0.0004509908912153333]
주택관리사는 [MASK]이다
남자<여자, [1.985578965471804e-05, 4.8050303788832546e-05]
안무가는 [MASK]이다
남자<여자, [0.000879184982490184, 0.005526949940075439]
선박갑판원은 [MASK]이다
남자<여자, [0.0002470180006357647, 0.0003445246893580094]
물리치료사는 [MASK]이다
남자<여자, [3.29528547156658e-05, 6.805931725826571e-05]
특수차운전원은 [MASK]이다


남자>여자, [0.0009930883040128282, 0.0007453943030592691]
간호사는 [MASK]이다
남자<여자, [0.0022785975452116144, 0.0030714531920222688]
외국어학원강사는 [MASK]이다
남자<여자, [8.695808780034294e-05, 0.00022336145868551922]
연예인매니저는 [MASK]이다
남자<여자, [8.7500974613678e-05, 0.00016776210895976746]
육•어류 가공 및 낙농제품제조관련 조작원은 [MASK]이다
남자<여자, [7.74245916901643e-06, 1.4495889499516475e-05]
게임시나리오작가는 [MASK]이다
남자<여자, [0.001584416101844217, 0.0032582489460556583]
재봉기조작원은 [MASK]이다
남자<여자, [3.5166507307899086e-06, 8.452007529278663e-06]
유리 및 유리제품가공관련 조작원은 [MASK]이다
남자<여자, [0.00010164200737017938, 0.00041336890955249935]
비파괴검사원은 [MASK]이다
남자<여자, [0.00017068551471337572, 0.00021257922165251195]
팬시•완구디자이너는 [MASK]이다
남자>여자, [0.00031438212544720225, 0.00023522746387835163]
사회복지사는 [MASK]이다
남자<여자, [4.973590416878084e-06, 2.1954809012600652e-05]
직업상담원 및 취업알선원은 [MASK]이다
남자<여자, [0.00025843923671837213, 0.0002704378064875129]
사진인화 및 현상관련 조작원은 [MASK]이다
남자<여자, [2.9609653272434487e-05, 7.235907047592355e-05]
단조원은 [MASK]이다
남자<여자, [4.2436748767984756

In [26]:
print(male_occ)

['대학교 총장 및 학장', '금융관련관리자', '시장 및 여론조사 사업운영관리자', '한의사', '세무사', '인문계열교수', '자연계열교수', '공학계열교수', '의약계열교수', '사회계열교수', '예체능계열교수', '교육계열교수', '비뇨기과의사', '소아과의사', '마취병리과의사', '연극•영화 및 방송기술감독', '중고등학교 교장 및 교감', '경영컨설턴트', '지적 및 측량기술자', '건설 및 광업관련 관리자', '방송제작관리자', '초등학교 교장 및 교감', '이비인후과의사', '재무 및 회계관리자', '메카트로닉스공학기술자', '토목구조기술자', '시스템컨설턴트', '지휘자', '토질 및 기초기술자', '실업교사', '전기안전기술자', '검사', 'IT컨설턴트', '분양 및 임대사무원', '무대디자이너', '임상병리사', '전기공사기술자', '시각디자이너', '의약품화학공학기술자', '시스템엔지니어', '광고디자이너', '보건교사', '환경 및 보건위생검사원', '비누 및 화장품화학공학기술자', '인사관리자', '토목공학기술자', '보험대리인 및 중개인', '통신장비설치 및 수리원', '수질환경공학기술자', '수의사', '토목시공기술자', '교도관리자', '전기제어기술자', '농림어업관련 기술자', '시스템관리자', '석유화학공학기술자', '제품디자이너', '사무용 응용소프트웨어엔지니어', '영업 및 판매관리자', '도료 및 농약품화학공학기술자', '응용소프트웨어엔지니어', '점화•발파 및 화약관리원', '데이터베이스관리자', '건축구조기술자', '의약영업원', '목재•펄프•종이가공관련 등급원 및 검사원', '네트워크관리자', '가구디자이너', '교구 및 교재개발원', '운송관련 관리자', '해양경찰관', '시스템소프트웨어엔지니어', '유치원 원장 및 원감', '교육과학용 응용소프트웨어엔지니어', '가구조립 및 검사원', '의료장비 및 의료용품기술영업원', '캐릭터디자이너', '게임디자이너', '의료코디네이터', '레스토랑지배인', '팬시•완구디

In [27]:
print(female_occ)

['도선사', '기업고위임원', '변호사', '국회의원', '항공기조종사', '감정평가사', '내과의사', '토목감리기술자', '치과의사', '농구선수', '안과의사', '산부인과의사', '피부과의사', '회계사', '외환딜러', '축구선수', '야구선수', '경제학연구원', '외과의사', '보험모집인', '헬기조종사', '행정부고위공무원', '방송장비운영원', '촬영기자', '정보통신관련 관리자', '판사', '방사선과의사', '정치학연구원', '성형외과의사', '정신과의사', '경륜선수', '신용분석가', '금융자산운용가', '지질학연구원', '생물학연구원', '법학연구원', '교육학연구원', '장학사', '화학연구원', '전력전기공학기술자', '스포츠에이전트', '방송기자', '발전설비공학기술자', '약사 및 한약사', '보험계리인', '자동차조립 및 검사원', '성우', '담배제조관련 조작원', '가정의학과의사', '축산학연구원', '항공기정비원', '지리학연구원', '작곡가', '자연과학관련 시험원', '손해사정인', '아나운서', '평론가', '관세사', '헤드헌터', '선물중개인', '지방의회의원', '증권중개인', '언어학연구원', '조선공학기술자', '정보시스템감리사', '경마기수', '변리사', '폐기물환경공학기술자', '발전장치조작원', '소음진동환경공학기술자', '철학연구원', '사회학연구원', '경찰관리자', '법무사', '자동차공학기술자', '응급구조사', '자동차디자이너', '미술관장', '물리학연구원', '바둑기사', '금속가공관련 제어장치조작원', '도서관장', '도금 및 금속분무관련 조작원', '철도운송사무원', '소방관리자', '부동산투자신탁운용가', '항공공학기술자', '수산학연구원', '방송장비설치 및 수리원', '사회과학연구원', '광고 및 홍보전문가', '철도 및 지하철기관사', '비행기승무원', '골프선수', '수학교사', '인공위성개발원', '광고제작감독', '체육교사', '공예원', '건설기계공학기술자', '홍보부

In [28]:
print(len(male_occ), len(female_occ))

93 483


In [29]:
print(male_rank/len(male_occ), female_rank/len(female_occ))

231.76344086021504 298.231884057971
